[Oregon Curriculum Network](http://www.4dsolutions.net/ocn) <br />
[Discovering Math with Python](Introduction.ipynb)


# Chapter 5: PUBLIC KEY CRYPTOGRAPHY

What we discuss below is a recent breakthrough in cryptography.  When this cryptosystem first appeared in the UK at GCHQ, it was kept secret, because the implications had not been worked out.  

However with the emergence of the web and the need for businesses to securely transact as strangers, a public key crytosystem became very appealing. Exchanging secret keys like we used to would not be nearly as convenient. We needed public key crypto to transact business over the web.

The MIT team with initials R, S, A, managed to keep the USG from clamping down on their patent, since expired.  [The history](https://goo.gl/MGLvXs) is pretty fascinating.  Don't forget to read up on PGP (Pretty Good Privacy).

## Euler's Theorem

We're now in a position to test Euler's Theorem, which is going to help us understand an important topic:  public key cryptography.

We saw with our permutation class (which still needs a \_\_pow\_\_ method by the way) how Bob and Alice might use what's called symmetric key cryptography.  This means Alice needs the same secret key Bob used, to encrypt a message, to decrypt it.

In public key cryptography, Bob and Alice both publish a public number, which in the RSA system is a really large number with only two prime factors, call it Bob_N and Alice_N.  

When Bob sends a message to Alice, he uses her public key, Alice_N, to encrypt it and only Alice has a corresponding secret key to get the message back.  Not even Bob can decrypt his own message.

Let's begin by understanding Euler's Theorem, which says:

$ a^{\varphi(n)} \equiv 1 \mod n$ where $\varphi$ is the totient function, and where a and n are relatively prime (co-prime, strangers).

In [1]:
import math

def totatives(n : int) -> list:
    """get co-primes to n between 0 and n"""
    return [totative for totative in range(n) if math.gcd(totative, n) == 1]

def totient(n):
    """how many totatives have we?"""
    return len(totatives(n))
    
print("Totient of 12:", totient(12))
print("Totient of 100:", totient(100))

Totient of 12: 4
Totient of 100: 40


In [2]:
pow(93, totient(100), 100)  # pow takes a modulus as an optional 3rd argument

1

The code cell below should always return 1, because we start with a co-prime of 100, satisfying the condition of Euler's Theorem.  The totient of 100 is always 40.

In [3]:
from random import choice
coprime = choice(totatives(100))
pow(coprime, 40, 100)

1

## RSA in a Nutshell

The basic idea of RSA is raising a message m to some power e, modulo N, turns it into an encrypted message c.  

Some special preparation is needed first, to randomly pad message m, a kind of seeding, otherwise some clever hacks may crack into the Bob-Alice transmission, allowing Eve to listen in.

Think of the message going part way around a circle when we raise it to the eth power.  We want it to go the rest of the way around the circle so that it pops out again, intact.  Given Euler's Theorem, one more power beyond the totient power will do the trick.  We need the inverse of c modulo that totient, in other words.  We'd looked at code for that already.

In other words, raising anything to the 1st power is equivalent to the original.  A zeroth power gives the multiplicative identity, typically. A\*\*0 == One, whatever One means in that namespace.  

Any power equivalent to 1, modulo totient(N), is our goal (thanks to Euler's Theorem), meaning to decrypt c after raising m to e mod N, we need some d such that (c * d) mod totient(N) == 1.  One notch beyond the totient power, going around in our circle, is the original m, where we started the whole process by raising by e. 

The message will pop right back out at exponent position (c * d), having gone all the way around the circle. Alice_d is what only Alice has, remember, when Bob uses Alice_N and Alice_e to make c, and send it, fairly secure in knowing that even if Eve gets the bits, she won't have a way to unscramble them (unless she's secretly the same person as Alice, but then mathematics is no protection against some forms of deception).

Here's the complete round-trip process, from Alice to Bob or Bob to Alice:

In [4]:
from groups import M
N = 3 * 47                  # generate a new N from scratch, per web browser session
m = M(42, N)                # 42 is original message, N a product of 2 primes
e = 7                       # raise to power, which others take as a signal how far to go
c = m ** e                  # encrypted
d = ~M(e, totient(N))       # inverse of c mod totient(3 * 47), kept private
pow(c, d.val)               # getting our message, using secret d

(42 mod 141)

42 is not a very exciting message to be sending to Alice however, plus our N is ridiculously small, very easy to factor.  What makes RSA hard to crack is we need those two prime factors of N to compute both its totient (as counting totatives won't do) and to thereby compute e's inverse i.e. d, the power we need to raise c by (mod N) to get our m back (the original message).

If you recall Chapter 4, our method for finding the inverse of an M-number involved using brute force.  We simply go through all the totatives until we find the right one.  Where huge Ns are involved, this technique is impractical. Is our whole cryptosystem just a pipe dream then?

### Euclid's Extended Method

 What comes to the rescue is called Euclid's Extended Method.  Lets look at that here.

In [5]:
def bingcd(a,b):
    """Extended version of Euclid's Algorithm (binary GCD)
    Returns (m,n,gcd) such that  m*a + n*b = gcd(a,b)"""
    g,u,v = [b,a],[1,0],[0,1]
    while g[1] != 0:
        y = g[0] // g[1]
        g[0],g[1] = g[1],g[0] % g[1]
        u[0],u[1] = u[1],u[0] - y*u[1]
        v[0],v[1] = v[1],v[0] - y*v[1]
    m = v[0]%b
    gcd = (m*a)%b
    n = (gcd - m*a)/b
    return (m,n,gcd)

def inverse(a,b):
    """
    If gcd(a,b)=1, then (inverse(a, b) * a) mod b = 1,
    otherwise, if gcd(a,b)!=1, return 0

    Useful in RSA encryption, for finding d such that
    e*d mod totient(n) == 1
    """
    inva,n,gcd = bingcd(a,b)
    return (gcd==1)*inva

print(inverse(7, 141))

121


Spending some quality time with both Euclid's Method (EA) for find the gcd, and the above more elaborate method (EEA) is always good practice.  Their coding has gotten very clever over the years, as coding languages have evolved and become more expressive.  However these methods have their historical roots in the more distant past, before people programmed in any computer language.

In [6]:
M(7, 141) * M(121, 141)

(1 mod 141)

Another important fact from Number Theory also comes to our rescue.  What is the totient of a number?  Counting totatives will take forever, practically, once N is large enough.  However, if N is comprised of two primes, p, q, then it will have (p-1)(q-1) totatives.  We can test this:

In [7]:
N = 47 * 19
(47-1)*(19 - 1) == totient(N)  # defined above

True

Why is this true?  47 has 46 totatives, 1 through 46, and 19 has 18 totatives.  Because they're prime numbers. Nothing less divides into them evenly (with no remainder) by definition, otherwise we would say they have factors, are composite. 

Their product will have like a multiplication table of these two sets, imagine 1-46 and 1-18 across the top and down the side, like a pandas Dataframe (see Chapter 1). All those products make up the newer, bigger list of totatives, of the newer bigger product number of only two factors.

Below is a published RSA-number from Wikipedia, already factored for us.  If N were that easy to crack open, with a quick lookup in Wikipedia, RSA would not still serve as a first line of defense against unauthorized Eves.

The RSA company held contests to encourage people to try cracking them, and some of them did crack, leading to the practice of yet longer numbers, which take exponentially longer to crack according to current theories.  The discovery of some algorithm to factor giant numbers in little time, would revolutionize our vista, as far as cryptosystems go.

## An Example

Here's that RSA number, named RSA-210:

In [8]:
p = 435958568325940791799951965387214406385470910265220196318705482144\
524085345275999740244625255428455944579
q = 5625457617268841037562770073044474\
81743876944007510545104946851094548396577479473472146228550799322939273

N = RSA210 =  p * q
t = (p-1)*(q-1)
d = ~M(3, t)     # need a d, such that 3*d mod t == 1
(3 * d.val) % t  # confirm our ability to find inverses with EEA

1

In [9]:
from binascii import hexlify, unhexlify

phrase = hexlify(b"Able was i ere i saw Elba")
phrase

b'41626c652077617320692065726520692073617720456c6261'

In [10]:
m = int(phrase, 16)
m

410424947989148738454394815364866651016047497091321737011809

In [11]:
m = M(m, RSA210)
c = m ** 3                     # encrypted
c

(69135523462041136185714565774079977390384666301915729148436060818425045084502570834616401113809997102784986010540370435543222688944984776423588149291933710092622905546285285348129 mod 245246644900278211976517663573088018467026787678332759743414451715061600830038587216952208399332071549103626827191679864079776723243005600592035631246561218465817904100131859299619933817012149335034875870551067)

In [12]:
result = pow(c.val, d.val, RSA210)
result

410424947989148738454394815364866651016047497091321737011809

In [13]:
unhexlify(hex(result)[2:])  # drop the leading 0x from output of hex()

b'Able was i ere i saw Elba'

So much for bare bones RSA.  If you want to develop a trully robust cryptosystem from scratch then do more reading outside the confines of this particular Notebook.  For example, 3 is probably not the best exponent to use at the outset.  

Indeed, your sender, Bob or Alice, needs to know what e you used, as that's what they need to raise theirs to.  (e, N) is more properly speaking your full public key.  Your d (for decrypt) is what you can't let out.  That's another vulnerability.

In other words, no warranty is expressed nor implied with regard to taking this code and using it within some mission-critical system.  That being said, have fun with it.

[Back to Chapter 4](Clock Arithmetic.ipynb)<br />
[Introduction](Introduction.ipynb)